In [2]:
# Cell 1: Import Libraries
import pandas as pd
import pyodbc
import numpy as np

# For basic text cleaning
from bs4 import BeautifulSoup # For removing HTML tags
import re # For regular expressions
import contractions # For expanding contractions (pip install contractions)
# For NLP tasks
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
print("Downloading required NLTK data...")
nltk.download('punkt') # Tokenizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4') # Open Multilingual Wordnet
print("NLTK data download complete.")

print("Libraries imported successfully.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...


NLTK data download complete.
Libraries imported successfully.


[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# Cell 2: Define Text Cleaning Function
# Initialize NLTK components
try:
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
except LookupError:
    print("NLTK resources not found. Please run nltk.download() for 'stopwords' and 'wordnet'.")
    raise

def clean_and_preprocess_text(text):
    # ... (the full function definition from previous instructions) ...
     if not isinstance(text, str):
         return ""
     text = text.lower()
     text = BeautifulSoup(text, "html.parser").get_text()
     text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
     emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"
         u"\U0001F300-\U0001F5FF"
         u"\U0001F680-\U0001F6FF"
         u"\U0001F1E0-\U0001F1FF"
         u"\U00002500-\U00002BEF"
         u"\U00002702-\U000027B0"
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         u"\U0001f926-\U0001f937"
         u"\U00010000-\U0010ffff"
         u"\u2640-\u2642"
         u"\u2600-\u2B55"
         u"\u200d"
         u"\u23cf"
         u"\u23e9"
         u"\u231a"
         u"\ufe0f"
         u"\u3030"
         "]+", flags=re.UNICODE)
     text = emoji_pattern.sub(r'', text)
     text = contractions.fix(text)
     text = re.sub(r'[^a-zA-Z\s]', '', text)
     tokens = word_tokenize(text)
     filtered_tokens = [word for word in tokens if word not in stop_words]
     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
     cleaned_text = ' '.join(lemmatized_tokens)
     return cleaned_text

In [5]:
# Cell 2: Load Data from SQL Server
# --- Database Connection Configuration (Update as needed) ---
SERVER = 'localhost' # Replace with your server name/IP
DATABASE = 'PlayStoreReviewsDB'
# Use Windows Authentication (common for local development)
CONNECTION_STRING = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'
# Or SQL Server Authentication (uncomment and update if needed):
# USERNAME = 'your_username'
# PASSWORD = 'your_password'
# CONNECTION_STRING = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

def load_data_from_db():
    """Loads data from the AppReviews table into a Pandas DataFrame."""
    try:
        conn = pyodbc.connect(CONNECTION_STRING)
        print("Connected to SQL Server successfully.")
        # Query to select all data from the AppReviews table
        query = "SELECT * FROM dbo.AppReviews"
        # Load data into a DataFrame
        df = pd.read_sql(query, conn)
        conn.close()
        print("Data loaded into DataFrame successfully.")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame() # Return empty DataFrame on error

# Load the data
df = load_data_from_db()

# Display basic information about the DataFrame
print("\n--- DataFrame Info ---")
print(df.info())

print("\n--- First 5 Rows ---")
print(df.head())

print("\n--- DataFrame Shape ---")
print(df.shape)

Connected to SQL Server successfully.
Data loaded into DataFrame successfully.

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ReviewID           1500 non-null   int64         
 1   AppName            1500 non-null   object        
 2   PlayStoreReviewID  1500 non-null   object        
 3   UserName           1500 non-null   object        
 4   UserImageURL       1500 non-null   object        
 5   ReviewText         1500 non-null   object        
 6   Rating             1500 non-null   int64         
 7   ThumbsUpCount      1500 non-null   int64         
 8   AppVersion         1373 non-null   object        
 9   ReviewDate         1500 non-null   datetime64[ns]
 10  ReplyContent       672 non-null    object        
 11  RepliedAt          672 non-null    datetime64[ns]
 12  ScrapedAt      

C:\Users\hp\AppData\Local\Temp\ipykernel_15368\4116662737.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
# Cell 3: Initial Data Inspection
print("--- Data Types ---")
print(df.dtypes)

print("\n--- Unique Apps ---")
print(df['AppName'].unique()) # Should match the apps you scraped

print("\n--- Rating Distribution ---")
print(df['Rating'].value_counts().sort_index()) # Count of each rating (1-5)

print("\n--- Missing Values ---")
print(df.isnull().sum()) # Count of missing values per column

print("\n--- Sample Review Texts ---")
# Look at a few examples, especially for columns that might need cleaning
print("Example ReviewText:")
print(df['ReviewText'].iloc[0])
print("\nAnother Example ReviewText:")
print(df['ReviewText'].iloc[1])

# Check for potential issues like empty strings
print(f"\nNumber of rows where ReviewText is NaN: {df['ReviewText'].isnull().sum()}")
print(f"Number of rows where ReviewText is empty string: {(df['ReviewText'] == '').sum()}")
print(f"Number of rows where ReviewText is only whitespace: {(df['ReviewText'].str.strip() == '').sum()}")

--- Data Types ---
ReviewID                      int64
AppName                      object
PlayStoreReviewID            object
UserName                     object
UserImageURL                 object
ReviewText                   object
Rating                        int64
ThumbsUpCount                 int64
AppVersion                   object
ReviewDate           datetime64[ns]
ReplyContent                 object
RepliedAt            datetime64[ns]
ScrapedAt            datetime64[ns]
dtype: object

--- Unique Apps ---
['AliExpress' 'Alibaba' 'Jiji']

--- Rating Distribution ---
Rating
1    536
2    127
3    134
4    256
5    447
Name: count, dtype: int64

--- Missing Values ---
ReviewID               0
AppName                0
PlayStoreReviewID      0
UserName               0
UserImageURL           0
ReviewText             0
Rating                 0
ThumbsUpCount          0
AppVersion           127
ReviewDate             0
ReplyContent         828
RepliedAt            828
ScrapedAt      

In [4]:
# --- Diagnostic Cell: Check DataFrame Structure ---
print("--- DataFrame Columns ---")
print(df.columns.tolist()) # This lists ALL column names exactly as they are

print("\n--- DataFrame Info ---")
print(df.info()) # This also shows column names and data types

print("\n--- First Few Rows (Transposed for easy column viewing) ---")
print(df.head().T) # Transposing makes columns easier to see

--- DataFrame Columns ---
['ReviewID', 'AppName', 'PlayStoreReviewID', 'UserName', 'UserImageURL', 'ReviewText', 'Rating', 'ThumbsUpCount', 'AppVersion', 'ReviewDate', 'ReplyContent', 'RepliedAt', 'ScrapedAt']

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ReviewID           1500 non-null   int64         
 1   AppName            1500 non-null   object        
 2   PlayStoreReviewID  1500 non-null   object        
 3   UserName           1500 non-null   object        
 4   UserImageURL       1500 non-null   object        
 5   ReviewText         1500 non-null   object        
 6   Rating             1500 non-null   int64         
 7   ThumbsUpCount      1500 non-null   int64         
 8   AppVersion         1373 non-null   object        
 9   ReviewDate         1500 non-null   datetime64[ns]

In [6]:
# --- Diagnostic Cell: Check DataFrame Structure ---
# Make sure this cell is run AFTER you load the data into 'df'
print("--- DataFrame Columns ---")
print(df.columns.tolist()) # This lists ALL column names exactly as they are

print("\n--- DataFrame Info ---")
print(df.info()) # This also shows column names and data types

# Let's also check the first few rows to be absolutely sure
print("\n--- First Few Rows ---")
print(df.head())

--- DataFrame Columns ---
['ReviewID', 'AppName', 'PlayStoreReviewID', 'UserName', 'UserImageURL', 'ReviewText', 'Rating', 'ThumbsUpCount', 'AppVersion', 'ReviewDate', 'ReplyContent', 'RepliedAt', 'ScrapedAt']

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ReviewID           1500 non-null   int64         
 1   AppName            1500 non-null   object        
 2   PlayStoreReviewID  1500 non-null   object        
 3   UserName           1500 non-null   object        
 4   UserImageURL       1500 non-null   object        
 5   ReviewText         1500 non-null   object        
 6   Rating             1500 non-null   int64         
 7   ThumbsUpCount      1500 non-null   int64         
 8   AppVersion         1373 non-null   object        
 9   ReviewDate         1500 non-null   datetime64[ns]

In [8]:
# --- Simple Diagnostic Test ---
# Run this cell immediately before the cell that fails
print("Running diagnostic test...")
print(f"Type of 'df': {type(df)}")
print(f"Shape of 'df': {df.shape}")
if 'ReviewText' in df.columns:
    print("SUCCESS: Column 'ReviewText' found in df.columns!")
    print(f"First ReviewText sample: {repr(df['ReviewText'].iloc[0])}") # repr shows hidden characters
else:
    print("ERROR: Column 'ReviewText' NOT found in df.columns!")
    print(f"Actual columns are: {df.columns.tolist()}")
print("Diagnostic test finished.")

Running diagnostic test...
Type of 'df': <class 'pandas.core.frame.DataFrame'>
Shape of 'df': (1500, 13)
SUCCESS: Column 'ReviewText' found in df.columns!
First ReviewText sample: 'Good prices but awful shipping fees. Difficult to track shipping progress on orders: no arrival estimations or delivery notifications. Products often come as incorrect or flawed. Floods inboxes with spam. Store page is inconvenient to navigate. Whatever item you\'re looking for will be in a compilation of things under generic names (such as "114A" or "blue person").'
Diagnostic test finished.


In [10]:
# Cell 4: Handle Missing Values and Data Types
print(f"Original DataFrame shape: {df.shape}")

# --- Handle Missing/Empty ReviewText ---
# Remove rows where ReviewText is null, empty, or just whitespace
initial_rows = df.shape[0]
df_cleaned = df[df['ReviewText'].notnull() & (df['ReviewText'].str.strip() != '')]
rows_after_text_clean = df_cleaned.shape[0]
print(f"Dropped {initial_rows - rows_after_text_clean} rows due to missing/empty ReviewText.")
print(f"DataFrame shape after text cleaning: {df_cleaned.shape}")

# --- Handle Data Types (if needed) ---
# Ensure ReviewDate is datetime (it likely already is from scraping)
# df_cleaned['ReviewDate'] = pd.to_datetime(df_cleaned['ReviewDate'])

# --- Optional: Fill other missing values with placeholders (example) ---
# df_cleaned['UserName'].fillna('Anonymous', inplace=True)
# df_cleaned['AppVersion'].fillna('Unknown', inplace=True)

# Reset index after dropping rows (good practice)
df_cleaned = df_cleaned.reset_index(drop=True)

print("\n--- Missing Values after cleaning ---")
print(df_cleaned.isnull().sum())

# Update the main DataFrame reference
df = df_cleaned
del df_cleaned # Free up memory reference

Original DataFrame shape: (1500, 13)
Dropped 0 rows due to missing/empty ReviewText.
DataFrame shape after text cleaning: (1500, 13)

--- Missing Values after cleaning ---
ReviewID               0
AppName                0
PlayStoreReviewID      0
UserName               0
UserImageURL           0
ReviewText             0
Rating                 0
ThumbsUpCount          0
AppVersion           127
ReviewDate             0
ReplyContent         828
RepliedAt            828
ScrapedAt              0
dtype: int64


In [11]:
# Cell 1 (or wherever you have the other downloads): Download NLTK Data
# ... (your existing nltk.download commands) ...
print("Downloading required NLTK data...")
nltk.download('punkt')       # Tokenizer models (older)
nltk.download('punkt_tab')   # NEW: Tokenizer models (newer, required)
nltk.download('stopwords')   # Stopwords list
nltk.download('wordnet')     # WordNet lexical database (for lemmatization)
nltk.download('omw-1.4')     # Open Multilingual Wordnet (needed for wordnet)
print("NLTK data download complete.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...


NLTK data download complete.


[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
# Cell 5: Define Text Cleaning Function
# Initialize NLTK components (do this once per session if not already done)
try:
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
except LookupError:
    print("NLTK resources not found. Please run nltk.download() for 'stopwords' and 'wordnet'.")
    raise

def clean_and_preprocess_text(text):
    """
    Cleans and preprocesses a single piece of text.
    Steps: Lowercasing, HTML removal, URL removal, emoji removal,
          punctuation removal, contraction expansion, tokenization,
          stopword removal, lemmatization.
    """
    if not isinstance(text, str):
         return "" # Return empty string for non-string inputs

    # 1. Lowercasing
    text = text.lower()

    # 2. Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # 3. Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # 4. Remove Emojis (basic regex approach)
    # More robust handling can use libraries like 'demoji'
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # 5. Expand Contractions (e.g., "don't" -> "do not")
    text = contractions.fix(text)

    # 6. Remove punctuation and numbers (keep only letters and spaces)
    # You might want to be more nuanced here (e.g., keep '$' for price mentions)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # 7. Tokenization
    tokens = word_tokenize(text)

    # 8. Remove Stopwords
    # Consider if domain-specific words like 'app', 'buy', 'price' should be kept
    # For now, we'll remove standard stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # 9. Lemmatization (reduces words to their base/dictionary form)
    # e.g., 'running', 'ran' -> 'run'
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # 10. Join tokens back into a single string
    cleaned_text = ' '.join(lemmatized_tokens)

    return cleaned_text

# Test the function on a sample
sample_text = df['ReviewText'].iloc[0]
print("Original Text:")
print(sample_text)
print("\nCleaned & Preprocessed Text:")
print(clean_and_preprocess_text(sample_text))

Original Text:
Good prices but awful shipping fees. Difficult to track shipping progress on orders: no arrival estimations or delivery notifications. Products often come as incorrect or flawed. Floods inboxes with spam. Store page is inconvenient to navigate. Whatever item you're looking for will be in a compilation of things under generic names (such as "114A" or "blue person").

Cleaned & Preprocessed Text:
good price awful shipping fee difficult track shipping progress order arrival estimation delivery notification product often come incorrect flawed flood inboxes spam store page inconvenient navigate whatever item looking compilation thing generic name blue person


In [13]:
# Cell 6: Apply Text Cleaning Function
print("Starting text cleaning and preprocessing...")
# Apply the function to the 'ReviewText' column
# This might take a minute or two depending on your dataset size
df['CleanedReviewText'] = df['ReviewText'].apply(clean_and_preprocess_text)
print("Text cleaning and preprocessing completed.")

# --- Optional: Feature Engineering ---
# Calculate review length (number of words in cleaned text)
df['ReviewWordCount'] = df['CleanedReviewText'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

# --- Inspect Results ---
print("\n--- Sample Original vs Cleaned Text ---")
print("Original:")
print(df['ReviewText'].iloc[0])
print("\nCleaned:")
print(df['CleanedReviewText'].iloc[0])
print(f"\nOriginal Word Count: {len(df['ReviewText'].iloc[0].split())}")
print(f"Cleaned Word Count: {df['ReviewWordCount'].iloc[0]}")

print("\n--- Another Sample ---")
print("Original:")
print(df['ReviewText'].iloc[1])
print("\nCleaned:")
print(df['CleanedReviewText'].iloc[1])

# Check for rows where cleaning resulted in empty strings
empty_after_cleaning = df[df['CleanedReviewText'] == '']
print(f"\nNumber of reviews that became empty after cleaning: {len(empty_after_cleaning)}")
if len(empty_after_cleaning) > 0:
    print("Dropping these rows...")
    df = df[df['CleanedReviewText'] != ''].reset_index(drop=True)
    print(f"Final DataFrame shape after dropping empty cleaned texts: {df.shape}")

Starting text cleaning and preprocessing...
Text cleaning and preprocessing completed.

--- Sample Original vs Cleaned Text ---
Original:
Good prices but awful shipping fees. Difficult to track shipping progress on orders: no arrival estimations or delivery notifications. Products often come as incorrect or flawed. Floods inboxes with spam. Store page is inconvenient to navigate. Whatever item you're looking for will be in a compilation of things under generic names (such as "114A" or "blue person").

Cleaned:
good price awful shipping fee difficult track shipping progress order arrival estimation delivery notification product often come incorrect flawed flood inboxes spam store page inconvenient navigate whatever item looking compilation thing generic name blue person

Original Word Count: 57
Cleaned Word Count: 35

--- Another Sample ---
Original:
Easy experience the last few years. Have done some shopping for tech parts you wouldn't really find in the US. I went up to buying figurin

In [5]:
# Cell 7: Save Cleaned Data
output_filename = 'cleaned_reviews_data.csv'
df.to_csv(output_filename, index=False) # Don't save the DataFrame index
print(f"Cleaned data saved to '{output_filename}'.")



Cleaned data saved to 'cleaned_reviews_data.csv'.


In [14]:
# --- Database Connection Configuration (Update as needed) ---
SERVER = 'localhost' # Replace with your server name/IP
DATABASE = 'PlayStoreReviewsDB'
CONNECTION_STRING = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'

def load_data_from_db():
    try:
        conn = pyodbc.connect(CONNECTION_STRING)
        print("Connected to SQL Server successfully.")
        query = "SELECT * FROM dbo.AppReviews"
        df = pd.read_sql(query, conn)
        conn.close()
        print("Data loaded into DataFrame successfully.")
        return df
    except Exception as e:
        print(f"Error loading  {e}")
        return pd.DataFrame()

# --- THIS LINE IS CRUCIAL ---
df = load_data_from_db() # <-- This assigns the loaded data to the variable 'df'

# Display basic information
print("\n--- DataFrame Info ---")
print(df.info())
print("\n--- First 5 Rows ---")
print(df.head())
print("\n--- DataFrame Shape ---")
print(df.shape)

Connected to SQL Server successfully.
Data loaded into DataFrame successfully.

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ReviewID           1500 non-null   int64         
 1   AppName            1500 non-null   object        
 2   PlayStoreReviewID  1500 non-null   object        
 3   UserName           1500 non-null   object        
 4   UserImageURL       1500 non-null   object        
 5   ReviewText         1500 non-null   object        
 6   Rating             1500 non-null   int64         
 7   ThumbsUpCount      1500 non-null   int64         
 8   AppVersion         1373 non-null   object        
 9   ReviewDate         1500 non-null   datetime64[ns]
 10  ReplyContent       672 non-null    object        
 11  RepliedAt          672 non-null    datetime64[ns]
 12  ScrapedAt      

C:\Users\hp\AppData\Local\Temp\ipykernel_5128\103105414.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [6]:
# Cell 6: Apply Text Cleaning Function
print("Starting text cleaning and preprocessing...")
# Apply the function to the 'ReviewText' column
# This might take a minute or two depending on your dataset size
df['CleanedReviewText'] = df['ReviewText'].apply(clean_and_preprocess_text)
print("Text cleaning and preprocessing completed.")

# --- Optional: Feature Engineering ---
# Calculate review length (number of words in cleaned text)
df['ReviewWordCount'] = df['CleanedReviewText'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

# --- Inspect Results ---
print("\n--- Sample Original vs Cleaned Text ---")
print("Original:")
print(df['ReviewText'].iloc[0])
print("\nCleaned:")
print(df['CleanedReviewText'].iloc[0])
print(f"\nOriginal Word Count: {len(df['ReviewText'].iloc[0].split())}")
print(f"Cleaned Word Count: {df['ReviewWordCount'].iloc[0]}")

print("\n--- Another Sample ---")
print("Original:")
print(df['ReviewText'].iloc[1])
print("\nCleaned:")
print(df['CleanedReviewText'].iloc[1])

# Check for rows where cleaning resulted in empty strings
empty_after_cleaning = df[df['CleanedReviewText'] == '']
print(f"\nNumber of reviews that became empty after cleaning: {len(empty_after_cleaning)}")
if len(empty_after_cleaning) > 0:
    print("Dropping these rows...")
    df = df[df['CleanedReviewText'] != ''].reset_index(drop=True)
    print(f"Final DataFrame shape after dropping empty cleaned texts: {df.shape}")

Starting text cleaning and preprocessing...
Text cleaning and preprocessing completed.

--- Sample Original vs Cleaned Text ---
Original:
Good prices but awful shipping fees. Difficult to track shipping progress on orders: no arrival estimations or delivery notifications. Products often come as incorrect or flawed. Floods inboxes with spam. Store page is inconvenient to navigate. Whatever item you're looking for will be in a compilation of things under generic names (such as "114A" or "blue person").

Cleaned:
good price awful shipping fee difficult track shipping progress order arrival estimation delivery notification product often come incorrect flawed flood inboxes spam store page inconvenient navigate whatever item looking compilation thing generic name blue person

Original Word Count: 57
Cleaned Word Count: 35

--- Another Sample ---
Original:
Easy experience the last few years. Have done some shopping for tech parts you wouldn't really find in the US. I went up to buying figurin

In [7]:
# Cell 7: Save Cleaned Data
output_filename = 'cleaned_reviews_data.csv'
df.to_csv(output_filename, index=False) # Don't save the DataFrame index
print(f"Cleaned data saved to '{output_filename}'.")

Cleaned data saved to 'cleaned_reviews_data.csv'.
